<a href="https://colab.research.google.com/github/thomngt/BigData_Project/blob/main/interface_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
0.## Kết nối Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)
folder = '/content/gdrive/My Drive/Colab Notebooks/Bigdata'

Mounted at /content/gdrive


In [ ]:
#Thư viện cần thiết:
import pandas as pd
import numpy as np
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
import re
import time
import tensorflow as tf
# Tắt những cảnh báo
import warnings
warnings.filterwarnings('ignore')

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import naive_bayes as nb
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
data=pd.read_csv(folder +'/DisneylandReviews.csv',encoding='ISO-8859-1')
data.head()

,Review_ID,Rating,Year_Month,Reviewer_Location,Review_Text,Branch
0,670772142,4,2019-4,Australia,If you've ever been to Disneyland anywhere you...,Disneyland_HongKong
1,670682799,4,2019-5,Philippines,Its been a while since d last time we visit HK...,Disneyland_HongKong
2,670623270,4,2019-4,United Arab Emirates,Thanks God it wasn t too hot or too humid wh...,Disneyland_HongKong
3,670607911,4,2019-4,Australia,HK Disneyland is a great compact park. Unfortu...,Disneyland_HongKong
4,670607296,4,2019-4,United Kingdom,"the location is not in the city, took around 1...",Disneyland_HongKong


Gắn nhãn cho cột Rating

In [ ]:
rating = data["Rating"]
def label_func(rating):
    if rating >=4:
        return "Positive"
    elif rating == 3:
            return "Neutral"
    else:
        return "Negative"

data["Sentiment"] = data["Rating"].apply(lambda x: label_func(x))

In [ ]:
#Tạo hàm tiền xử lý dữ liệu
stop_words = set(stopwords.words('english'))

def clean_reviews(text):
    # Xóa html
    regex = re.compile('<.*?>')
    text = re.sub(regex, '', text)

    # Xóa những ký tự đặc biệt
    pattern = re.compile('[^a-zA-z0-9\s]')
    text = re.sub(pattern, '', text)

    # Xóa chữ số
    pattern = re.compile('\d+')
    text = re.sub(pattern, '', text)

    # Chuyển đổi text sang chữ thường
    text = text.lower()

    # Tokenization of words
    text = word_tokenize(text)

    # Lemmatization of words
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word) for word in text]

    # Stop words removal
    text = [word for word in text if word not in stop_words]

    # Nối các từ tạo thành một chuỗi
    text = ' '.join(text)

    return text
data['Review_Text'] = data['Review_Text'].apply(clean_reviews)

In [ ]:
#data sau khi tiền xử lý
df_main_class = data[["Review_Text","Sentiment"]]
df_main_class

,Review_Text,Sentiment
0,youve ever disneyland anywhere youll find disn...,Positive
1,since last time visit hk disneyland yet time s...,Positive
2,thanks god hot humid wa visiting park otherwis...,Positive
3,hk disneyland great compact park unfortunately...,Positive
4,location city took around hour kowlon kid like...,Positive
...,...,...
42651,went disneyland paris july thought wa brillian...,Positive
42652,adult child visited disneyland paris beginning...,Positive
42653,eleven year old daughter went visit son london...,Positive
42654,hotel part disneyland paris complex wonderful ...,Positive


In [ ]:
X = df_main_class['Review_Text']
y = df_main_class['Sentiment']

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra với tỷ lệ 80-20
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, stratify= y, random_state=42)

print('Tập Train (80%): ', train_x.shape)
print('Tập Test (20%: ', test_x.shape)

Tập Train (80%):  (34124,)
Tập Test (20%:  (8532,)


In [ ]:
!pip install -q comet_ml gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 634.0/634.0 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.7/514.7 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 14

In [ ]:
!pip install -q comet_ml gradio
import gradio as gr
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# Hàm huấn luyện mô hình
def train_model(train_text, train_labels):
   # Tạo một vectorizer để chuyển đổi văn bản thành biểu diễn BoW
    vectorizer = CountVectorizer(analyzer='word', max_features=20000, stop_words='english')
    # Huấn luyện vectorizer trên dữ liệu huấn luyện
    vectorizer.fit(df_main_class['Review_Text'])
# Chuyển đổi dữ liệu huấn luyện và kiểm tra thành biểu diễn BoW
    xtrain_bow = vectorizer.transform(train_x)
    xtest_bow = vectorizer.transform(test_x)
   # Khởi tạo và huấn luyện mô hình logistic regression
    LogReg = LogisticRegression(penalty='l2', C=1.0, solver='lbfgs', max_iter=100)
    LogReg.fit(xtrain_bow, train_y)
  # Trả về mô hình đã được huấn luyện và vectorizer
    return LogReg, vectorizer

# Hàm dự đoán cảm xúc
def predict_sentiment(text, model, vectorizer):
  # Chuyển đổi văn bản thành biểu diễn BoW bằng cách sử dụng vectorizer
    text_vectorized = vectorizer.transform([text])
  # Dự đoán cảm xúc bằng cách sử dụng mô hình đã được huấn luyện
    prediction = model.predict(text_vectorized)[0]
  # Trả về dự đoán
    return prediction

# Huấn luyện mô hình
train_text = df_main_class['Review_Text']
train_labels = df_main_class['Sentiment']
model, vectorizer = train_model(train_text, train_labels)

# Giao diện Gradio
def classify_sentiment(text):
    prediction = predict_sentiment(text, model, vectorizer)
    return prediction

gr.Interface(fn=classify_sentiment, inputs="text", outputs="text").launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://21a647626b0978bfa1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
